In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')  
import seaborn as sns
from ydata_profiling import ProfileReport
df = pd.read_csv('Walmart.csv')
profile = ProfileReport(
    df,
    title="Walmart Stores Sales Data Profiling "
)
profile.to_file("Walmart Stores Sales Data.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 8/8 [00:00<00:00, 133.05it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
df.duplicated().sum()

0

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   float64
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6   CPI           6435 non-null   float64
 7   Unemployment  6435 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 402.3+ KB


In [26]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

In [27]:
# Extract useful time features
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Quarter'] = df['Date'].dt.quarter
df['IsWeekend'] = df['Date'].dt.dayofweek.isin([5, 6]).astype(int)

In [28]:
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    outlier_percentage = (len(outliers) / len(df)) * 100
    return outliers, lower_bound, upper_bound, outlier_percentage

numerical_cols = ['Weekly_Sales', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']

outlier_summary = {}
for col in numerical_cols:
    outliers, lower, upper, pct = detect_outliers_iqr(df, col)
    outlier_summary[col] = {
        'count': len(outliers),
        'percentage': pct,
        'lower_bound': lower,
        'upper_bound': upper
    }
    
    print(f"\n{col}:")
    print(f"  Outliers Count: {len(outliers)}")
    print(f"  Outliers Percentage: {pct:.2f}%")
    print(f"  Valid Range: [{lower:.2f}, {upper:.2f}]")
    print(f"  Min Value: {df[col].min():.2f}")
    print(f"  Max Value: {df[col].max():.2f}")


Weekly_Sales:
  Outliers Count: 34
  Outliers Percentage: 0.53%
  Valid Range: [-746862.73, 2720371.49]
  Min Value: 209986.25
  Max Value: 3818686.45

Temperature:
  Outliers Count: 3
  Outliers Percentage: 0.05%
  Valid Range: [6.24, 116.16]
  Min Value: -2.06
  Max Value: 100.14

Fuel_Price:
  Outliers Count: 0
  Outliers Percentage: 0.00%
  Valid Range: [1.73, 4.94]
  Min Value: 2.47
  Max Value: 4.47

CPI:
  Outliers Count: 0
  Outliers Percentage: 0.00%
  Valid Range: [10.22, 334.26]
  Min Value: 126.06
  Max Value: 227.23

Unemployment:
  Outliers Count: 481
  Outliers Percentage: 7.47%
  Valid Range: [4.29, 11.22]
  Min Value: 3.88
  Max Value: 14.31


In [29]:
# STEP 2: VISUALIZE OUTLIERS
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, col in enumerate(numerical_cols):
    df.boxplot(column=col, ax=axes[idx])
    pct = outlier_summary[col]['percentage']
    axes[idx].set_title(f'{col}\nOutliers: {pct:.1f}%')
    axes[idx].set_ylabel(col)
    
plt.tight_layout()
plt.show()

# Distribution plots to see outliers in context
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, col in enumerate(numerical_cols):
    axes[idx].hist(df[col], bins=50, edgecolor='black', alpha=0.7)
    axes[idx].axvline(outlier_summary[col]['lower_bound'], 
                      color='r', linestyle='--', label='Lower Bound')
    axes[idx].axvline(outlier_summary[col]['upper_bound'], 
                      color='r', linestyle='--', label='Upper Bound')
    axes[idx].set_title(f'{col} Distribution')
    axes[idx].set_xlabel(col)
    axes[idx].legend()
    
plt.tight_layout()
plt.show()

In [30]:
lower, upper = outlier_summary['Temperature']['lower_bound'], outlier_summary['Temperature']['upper_bound']
df['Temperature'] = df['Temperature'].clip(lower=lower, upper=upper)
lower_bound_unemp,upper_bound_unemp=outlier_summary['Unemployment']['lower_bound'],outlier_summary['Unemployment']['upper_bound']
df['Unemployment'] = df['Unemployment'].clip(lower=lower_bound_unemp, upper=upper_bound_unemp)
df['Log_Weekly_Sales'] = np.log1p(df['Weekly_Sales'])

In [31]:
df.head().T

,0,1,2,3,4
Store,1,1,1,1,1
Date,2010-02-05 00:00:00,2010-02-12 00:00:00,2010-02-19 00:00:00,2010-02-26 00:00:00,2010-03-05 00:00:00
Weekly_Sales,1643690.9,1641957.44,1611968.17,1409727.59,1554806.68
Holiday_Flag,0,1,0,0,0
Temperature,42.31,38.51,39.93,46.63,46.5
Fuel_Price,2.572,2.548,2.514,2.561,2.625
CPI,211.096358,211.24217,211.289143,211.319643,211.350143
Unemployment,8.106,8.106,8.106,8.106,8.106
Year,2010,2010,2010,2010,2010
Month,2,2,2,2,3


In [32]:
#EDA
#Univariate Analysis
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(df['Weekly_Sales'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
plt.title('Distribution of Weekly Sales', fontsize=14, fontweight='bold')
plt.xlabel('Weekly Sales ($)')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.3)

plt.subplot(1, 3, 2)
df['Weekly_Sales'].plot(kind='box', color='steelblue')
plt.title('Boxplot of Weekly Sales', fontsize=14, fontweight='bold')
plt.ylabel('Weekly Sales ($)')
plt.grid(axis='y', alpha=0.3)

plt.subplot(1, 3, 3)
df['Weekly_Sales'].plot(kind='kde', color='steelblue', linewidth=2)
plt.title('Density Plot of Weekly Sales', fontsize=14, fontweight='bold')
plt.xlabel('Weekly Sales ($)')
plt.ylabel('Density')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()


In [33]:
if 'Log_Weekly_Sales' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Original vs Log comparison
    axes[0].hist(df['Weekly_Sales'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    axes[0].set_title('Weekly Sales (Original Scale)', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Weekly Sales ($)')
    axes[0].set_ylabel('Frequency')
    axes[0].grid(axis='y', alpha=0.3)
    
    axes[1].hist(df['Log_Weekly_Sales'], bins=50, edgecolor='black', alpha=0.7, color='coral')
    axes[1].set_title('Log Weekly Sales', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Log(Weekly Sales)')
    axes[1].set_ylabel('Frequency')
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [34]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Holiday Flag
holiday_counts = df['Holiday_Flag'].value_counts().sort_index()
axes[0].bar(range(len(holiday_counts)), holiday_counts.values, color=['lightblue', 'coral'][:len(holiday_counts)])
axes[0].set_title('Holiday Flag Distribution', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Holiday Flag')
axes[0].set_ylabel('Count')
axes[0].set_xticks(range(len(holiday_counts)))
if len(holiday_counts) == 2:
    axes[0].set_xticklabels(['Non-Holiday (0)', 'Holiday (1)'], rotation=0)
else:
    axes[0].set_xticklabels([f'{idx}' for idx in holiday_counts.index], rotation=0)
# Top 10 Stores
df['Store'].value_counts().head(10).plot(kind='barh', ax=axes[1], color='steelblue')
axes[1].set_title('Top 10 Stores (Record Count)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Number of Records')
axes[1].set_ylabel('Store')

plt.tight_layout()
plt.show()


In [35]:
#Bivariate Analysis
# SALES BY HOLIDAY FLAG
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Average sales
holiday_avg = df.groupby('Holiday_Flag')['Weekly_Sales'].mean()
axes[0].bar(['Non-Holiday', 'Holiday'], holiday_avg.values, color=['skyblue', 'coral'], alpha=0.8)
axes[0].set_title('Average Sales: Holiday vs Non-Holiday', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Average Weekly Sales ($)')
axes[0].grid(axis='y', alpha=0.3)

# Add value labels
for i, v in enumerate(holiday_avg.values):
    axes[0].text(i, v, f'${v:,.0f}', ha='center', va='bottom', fontweight='bold')

# Distribution comparison
df.boxplot(column='Weekly_Sales', by='Holiday_Flag', ax=axes[1])
axes[1].set_title('Sales Distribution by Holiday Flag', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Holiday Flag (0=No, 1=Yes)')
axes[1].set_ylabel('Weekly Sales ($)')
axes[1].set_xticklabels(['Non-Holiday', 'Holiday'])
plt.suptitle('')

plt.tight_layout()
plt.show()

In [36]:
#  SALES BY WEEKEND
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

weekend_avg = df.groupby('IsWeekend')['Weekly_Sales'].mean()

# Check how many categories exist
num_categories = len(weekend_avg)

# Bar chart
if num_categories == 2:
    labels = ['Weekday', 'Weekend']
else:
    # Only one category exists
    labels = ['Weekday' if weekend_avg.index[0] == 0 else 'Weekend']

axes[0].bar(range(num_categories), weekend_avg.values, 
            color=['lightgreen', 'gold'][:num_categories], alpha=0.8)
axes[0].set_title('Average Sales: Weekday vs Weekend', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Average Weekly Sales ($)')
axes[0].set_xticks(range(num_categories))
axes[0].set_xticklabels(labels)
axes[0].grid(axis='y', alpha=0.3)

for i, v in enumerate(weekend_avg.values):
    axes[0].text(i, v, f'${v:,.0f}', ha='center', va='bottom', fontweight='bold')

# Boxplot
df.boxplot(column='Weekly_Sales', by='IsWeekend', ax=axes[1])
axes[1].set_title('Sales Distribution by Weekend', fontsize=14, fontweight='bold')
axes[1].set_xlabel('IsWeekend')
axes[1].set_ylabel('Weekly Sales ($)')

# Set x-axis labels based on actual data
unique_weekend_vals = df['IsWeekend'].unique()
if len(unique_weekend_vals) == 2:
    axes[1].set_xticklabels(['Weekday', 'Weekend'])
elif unique_weekend_vals[0] == 0:
    axes[1].set_xticklabels(['Weekday'])
else:
    axes[1].set_xticklabels(['Weekend'])

plt.suptitle('')
plt.tight_layout()
plt.show()

In [37]:
# ============================================================================
# TIME SERIES ANALYSIS - COMPREHENSIVE & CLEAR
# ============================================================================

print("="*70)
print("TIME SERIES ANALYSIS: WALMART SALES PATTERNS")
print("="*70)

# Ensure data is sorted by date
df_sorted = df.sort_values('Date').copy()

print(f"\nDate Range: {df_sorted['Date'].min().date()} to {df_sorted['Date'].max().date()}")
print(f"Total Time Span: {(df_sorted['Date'].max() - df_sorted['Date'].min()).days} days")
print(f"Number of Weeks: {df_sorted['Date'].nunique()}")
print(f"Number of Stores: {df_sorted['Store'].nunique()}")

# ============================================================================
# 1. OVERALL TREND - ALL STORES COMBINED
# ============================================================================

print("\n" + "-"*70)
print("1. OVERALL SALES TREND (ALL STORES)")
print("-"*70)

# Aggregate sales by date (sum across all stores)
daily_sales = df_sorted.groupby('Date')['Weekly_Sales'].sum().reset_index()
daily_sales_mean = df_sorted.groupby('Date')['Weekly_Sales'].mean().reset_index()

fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Total sales across all stores
axes[0].plot(daily_sales['Date'], daily_sales['Weekly_Sales'], 
             color='steelblue', linewidth=1.5, alpha=0.7)
axes[0].set_title('Total Weekly Sales Across All Stores', fontsize=16, fontweight='bold')
axes[0].set_xlabel('Date', fontsize=12)
axes[0].set_ylabel('Total Sales ($)', fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].axhline(daily_sales['Weekly_Sales'].mean(), color='red', linestyle='--', 
                linewidth=2, label=f"Average: ${daily_sales['Weekly_Sales'].mean():,.0f}")
axes[0].legend(fontsize=11)

# Add trend line
from scipy import stats
x_numeric = (daily_sales['Date'] - daily_sales['Date'].min()).dt.days
slope, intercept, r_value, p_value, std_err = stats.linregress(x_numeric, daily_sales['Weekly_Sales'])
trend_line = slope * x_numeric + intercept
axes[0].plot(daily_sales['Date'], trend_line, 'r-', linewidth=3, 
             alpha=0.5, label=f'Trend (slope={slope:,.0f}$/day)')
axes[0].legend(fontsize=11)

# Average sales per store
axes[1].plot(daily_sales_mean['Date'], daily_sales_mean['Weekly_Sales'], 
             color='coral', linewidth=1.5, alpha=0.7)
axes[1].set_title('Average Weekly Sales Per Store', fontsize=16, fontweight='bold')
axes[1].set_xlabel('Date', fontsize=12)
axes[1].set_ylabel('Average Sales ($)', fontsize=12)
axes[1].grid(True, alpha=0.3)
axes[1].axhline(daily_sales_mean['Weekly_Sales'].mean(), color='red', linestyle='--', 
                linewidth=2, label=f"Overall Avg: ${daily_sales_mean['Weekly_Sales'].mean():,.0f}")
axes[1].legend(fontsize=11)

plt.tight_layout()
plt.savefig('time_series_overall_trend.png', dpi=300, bbox_inches='tight')
plt.show()

# Print trend analysis
if slope > 0:
    trend_direction = "📈 UPWARD TREND"
    interpretation = "Sales are increasing over time"
elif slope < 0:
    trend_direction = "📉 DOWNWARD TREND"
    interpretation = "Sales are decreasing over time"
else:
    trend_direction = "➡️ FLAT TREND"
    interpretation = "Sales are stable over time"

print(f"\nTrend Analysis:")
print(f"  {trend_direction}: {interpretation}")
print(f"  Slope: ${slope:,.2f} per day")
print(f"  R² Score: {r_value**2:.4f} (how well trend line fits)")

# ============================================================================
# 2. YEARLY ANALYSIS
# ============================================================================

print("\n" + "-"*70)
print("2. YEARLY SALES ANALYSIS")
print("-"*70)

yearly_sales = df.groupby('Year')['Weekly_Sales'].agg([
    ('Total_Sales', 'sum'),
    ('Average_Sales', 'mean'),
    ('Median_Sales', 'median'),
    ('Std_Sales', 'std'),
    ('Min_Sales', 'min'),
    ('Max_Sales', 'max'),
    ('Count', 'count')
]).reset_index()

print("\nYearly Sales Summary:")
print(yearly_sales.to_string(index=False))

# Calculate year-over-year growth
if len(yearly_sales) > 1:
    print("\nYear-over-Year Growth:")
    for i in range(1, len(yearly_sales)):
        prev_year = yearly_sales.iloc[i-1]
        curr_year = yearly_sales.iloc[i]
        growth = ((curr_year['Total_Sales'] - prev_year['Total_Sales']) / prev_year['Total_Sales']) * 100
        print(f"  {int(prev_year['Year'])} → {int(curr_year['Year'])}: {growth:+.2f}%")

# Visualize yearly trends
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Total sales by year
axes[0, 0].bar(yearly_sales['Year'], yearly_sales['Total_Sales'], 
               color=['#667eea', '#764ba2', '#f093fb'][:len(yearly_sales)], alpha=0.8)
axes[0, 0].set_title('Total Sales by Year', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Year')
axes[0, 0].set_ylabel('Total Sales ($)')
axes[0, 0].grid(axis='y', alpha=0.3)
for i, v in enumerate(yearly_sales['Total_Sales']):
    axes[0, 0].text(yearly_sales['Year'].iloc[i], v, f'${v/1e9:.2f}B', 
                    ha='center', va='bottom', fontweight='bold')

# Average sales by year
axes[0, 1].bar(yearly_sales['Year'], yearly_sales['Average_Sales'], 
               color=['#4caf50', '#ff9800', '#f44336'][:len(yearly_sales)], alpha=0.8)
axes[0, 1].set_title('Average Weekly Sales by Year', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Year')
axes[0, 1].set_ylabel('Average Sales ($)')
axes[0, 1].grid(axis='y', alpha=0.3)
for i, v in enumerate(yearly_sales['Average_Sales']):
    axes[0, 1].text(yearly_sales['Year'].iloc[i], v, f'${v:,.0f}', 
                    ha='center', va='bottom', fontweight='bold')

# Monthly average across all years
monthly_avg = df.groupby('Month')['Weekly_Sales'].mean().reset_index()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
monthly_avg['Month_Name'] = monthly_avg['Month'].apply(lambda x: month_names[x-1])

axes[1, 0].plot(monthly_avg['Month'], monthly_avg['Weekly_Sales'], 
                marker='o', linewidth=3, markersize=10, color='steelblue')
axes[1, 0].set_title('Average Sales by Month (All Years)', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Average Sales ($)')
axes[1, 0].set_xticks(range(1, 13))
axes[1, 0].set_xticklabels(month_names, rotation=45)
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(monthly_avg['Weekly_Sales'].mean(), color='red', 
                   linestyle='--', linewidth=2, alpha=0.7)

# Sales distribution by year (box plot)
df.boxplot(column='Weekly_Sales', by='Year', ax=axes[1, 1])
axes[1, 1].set_title('Sales Distribution by Year', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Year')
axes[1, 1].set_ylabel('Weekly Sales ($)')
plt.suptitle('')

plt.tight_layout()
plt.savefig('time_series_yearly_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# 3. MONTHLY SEASONALITY ANALYSIS
# ============================================================================

print("\n" + "-"*70)
print("3. MONTHLY SEASONALITY PATTERNS")
print("-"*70)

monthly_stats = df.groupby('Month')['Weekly_Sales'].agg([
    ('Average', 'mean'),
    ('Median', 'median'),
    ('Std', 'std'),
    ('Min', 'min'),
    ('Max', 'max')
]).reset_index()
monthly_stats['Month_Name'] = monthly_stats['Month'].apply(lambda x: month_names[x-1])

print("\nMonthly Sales Statistics:")
print(monthly_stats[['Month_Name', 'Average', 'Median', 'Std']].to_string(index=False))

# Identify best and worst months
best_month_idx = monthly_stats['Average'].idxmax()
worst_month_idx = monthly_stats['Average'].idxmin()
best_month = monthly_stats.iloc[best_month_idx]
worst_month = monthly_stats.iloc[worst_month_idx]

print(f"\n📈 BEST MONTH: {best_month['Month_Name']}")
print(f"   Average Sales: ${best_month['Average']:,.2f}")
print(f"   {((best_month['Average'] - monthly_stats['Average'].mean()) / monthly_stats['Average'].mean() * 100):+.1f}% vs overall average")

print(f"\n📉 WORST MONTH: {worst_month['Month_Name']}")
print(f"   Average Sales: ${worst_month['Average']:,.2f}")
print(f"   {((worst_month['Average'] - monthly_stats['Average'].mean()) / monthly_stats['Average'].mean() * 100):+.1f}% vs overall average")

# Monthly heatmap by year
fig, axes = plt.subplots(2, 1, figsize=(16, 12))

# Line plot with error bars
axes[0].errorbar(monthly_stats['Month'], monthly_stats['Average'], 
                 yerr=monthly_stats['Std'], marker='o', markersize=10, 
                 linewidth=3, capsize=5, color='steelblue', 
                 ecolor='lightblue', alpha=0.8)
axes[0].fill_between(monthly_stats['Month'], 
                      monthly_stats['Average'] - monthly_stats['Std'],
                      monthly_stats['Average'] + monthly_stats['Std'],
                      alpha=0.2, color='steelblue')
axes[0].set_title('Monthly Sales Pattern with Variability (Mean ± Std)', 
                  fontsize=16, fontweight='bold')
axes[0].set_xlabel('Month', fontsize=12)
axes[0].set_ylabel('Average Sales ($)', fontsize=12)
axes[0].set_xticks(range(1, 13))
axes[0].set_xticklabels(month_names, rotation=45)
axes[0].grid(True, alpha=0.3)
axes[0].axhline(monthly_stats['Average'].mean(), color='red', 
                linestyle='--', linewidth=2, label='Overall Average')
axes[0].legend(fontsize=11)

# Heatmap: Month vs Year
pivot_monthly = df.pivot_table(values='Weekly_Sales', 
                                index='Month', 
                                columns='Year', 
                                aggfunc='mean')

im = axes[1].imshow(pivot_monthly.values, cmap='YlOrRd', aspect='auto')
axes[1].set_title('Sales Heatmap: Month vs Year', fontsize=16, fontweight='bold')
axes[1].set_xlabel('Year', fontsize=12)
axes[1].set_ylabel('Month', fontsize=12)
axes[1].set_xticks(range(len(pivot_monthly.columns)))
axes[1].set_xticklabels(pivot_monthly.columns)
axes[1].set_yticks(range(12))
axes[1].set_yticklabels(month_names)

# Add colorbar
cbar = plt.colorbar(im, ax=axes[1])
cbar.set_label('Average Sales ($)', fontsize=11)

# Add text annotations
for i in range(len(pivot_monthly.index)):
    for j in range(len(pivot_monthly.columns)):
        text = axes[1].text(j, i, f'${pivot_monthly.iloc[i, j]/1000:.0f}K',
                           ha="center", va="center", color="black", fontsize=9)

plt.tight_layout()
plt.savefig('time_series_monthly_seasonality.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# 4. QUARTERLY ANALYSIS
# ============================================================================

print("\n" + "-"*70)
print("4. QUARTERLY SALES PATTERNS")
print("-"*70)

quarterly_stats = df.groupby('Quarter')['Weekly_Sales'].agg([
    ('Average', 'mean'),
    ('Total', 'sum'),
    ('Count', 'count')
]).reset_index()

print("\nQuarterly Sales Statistics:")
print(quarterly_stats.to_string(index=False))

# Calculate which quarter is best
best_quarter = quarterly_stats.loc[quarterly_stats['Average'].idxmax(), 'Quarter']
worst_quarter = quarterly_stats.loc[quarterly_stats['Average'].idxmin(), 'Quarter']

print(f"\n🏆 BEST PERFORMING QUARTER: Q{best_quarter}")
print(f"   Average Sales: ${quarterly_stats.loc[quarterly_stats['Quarter']==best_quarter, 'Average'].values[0]:,.2f}")

print(f"\n📉 WORST PERFORMING QUARTER: Q{worst_quarter}")
print(f"   Average Sales: ${quarterly_stats.loc[quarterly_stats['Quarter']==worst_quarter, 'Average'].values[0]:,.2f}")

# Visualize quarters
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Average by quarter
colors_q = ['#e3f2fd', '#90caf9', '#42a5f5', '#1976d2']
axes[0].bar(quarterly_stats['Quarter'], quarterly_stats['Average'], 
            color=colors_q, alpha=0.8, edgecolor='black', linewidth=2)
axes[0].set_title('Average Sales by Quarter', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Quarter', fontsize=12)
axes[0].set_ylabel('Average Sales ($)', fontsize=12)
axes[0].set_xticks([1, 2, 3, 4])
axes[0].set_xticklabels(['Q1\n(Jan-Mar)', 'Q2\n(Apr-Jun)', 'Q3\n(Jul-Sep)', 'Q4\n(Oct-Dec)'])
axes[0].grid(axis='y', alpha=0.3)
for i, v in enumerate(quarterly_stats['Average']):
    axes[0].text(quarterly_stats['Quarter'].iloc[i], v, f'${v:,.0f}', 
                ha='center', va='bottom', fontweight='bold', fontsize=11)

# Quarterly trend by year
quarterly_yearly = df.groupby(['Year', 'Quarter'])['Weekly_Sales'].mean().unstack()
quarterly_yearly.plot(kind='bar', ax=axes[1], width=0.8, colormap='viridis')
axes[1].set_title('Average Sales by Quarter and Year', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Year', fontsize=12)
axes[1].set_ylabel('Average Sales ($)', fontsize=12)
axes[1].legend(title='Quarter', labels=['Q1', 'Q2', 'Q3', 'Q4'], fontsize=10)
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)
axes[1].grid(axis='y', alpha=0.3)

# Box plot by quarter
df.boxplot(column='Weekly_Sales', by='Quarter', ax=axes[2])
axes[2].set_title('Sales Distribution by Quarter', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Quarter', fontsize=12)
axes[2].set_ylabel('Weekly Sales ($)', fontsize=12)
axes[2].set_xticklabels(['Q1\n(Jan-Mar)', 'Q2\n(Apr-Jun)', 'Q3\n(Jul-Sep)', 'Q4\n(Oct-Dec)'])
plt.suptitle('')

plt.tight_layout()
plt.savefig('time_series_quarterly_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# 5. WEEK-OF-YEAR ANALYSIS (52 weeks)
# ============================================================================

print("\n" + "-"*70)
print("5. WEEK-OF-YEAR PATTERNS (DETAILED)")
print("-"*70)

# Create week of year if not exists
if 'WeekOfYear' not in df.columns:
    df['WeekOfYear'] = df['Date'].dt.isocalendar().week

weekly_pattern = df.groupby('WeekOfYear')['Weekly_Sales'].mean().reset_index()

# Identify peak weeks
top_5_weeks = weekly_pattern.nlargest(5, 'Weekly_Sales')
bottom_5_weeks = weekly_pattern.nsmallest(5, 'Weekly_Sales')

print("\nTop 5 Weeks (Highest Average Sales):")
for idx, row in top_5_weeks.iterrows():
    print(f"  Week {int(row['WeekOfYear'])}: ${row['Weekly_Sales']:,.2f}")

print("\nBottom 5 Weeks (Lowest Average Sales):")
for idx, row in bottom_5_weeks.iterrows():
    print(f"  Week {int(row['WeekOfYear'])}: ${row['Weekly_Sales']:,.2f}")

# Plot weekly pattern
plt.figure(figsize=(18, 6))
plt.plot(weekly_pattern['WeekOfYear'], weekly_pattern['Weekly_Sales'], 
         linewidth=2, color='steelblue', marker='o', markersize=4, alpha=0.7)
plt.title('Average Sales by Week of Year (52 Weeks)', fontsize=16, fontweight='bold')
plt.xlabel('Week of Year', fontsize=12)
plt.ylabel('Average Sales ($)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.axhline(weekly_pattern['Weekly_Sales'].mean(), color='red', 
            linestyle='--', linewidth=2, label='Average')

# Highlight holiday weeks (commonly weeks 47-52 and 1-2)
holiday_weeks = [47, 48, 49, 50, 51, 52, 1, 2]
for week in holiday_weeks:
    if week in weekly_pattern['WeekOfYear'].values:
        sales = weekly_pattern[weekly_pattern['WeekOfYear']==week]['Weekly_Sales'].values[0]
        plt.axvline(x=week, color='orange', linestyle=':', alpha=0.5)
plt.axvline(x=47, color='orange', linestyle=':', alpha=0.7, 
            label='Holiday Season', linewidth=2)

# Mark quarters
plt.axvline(x=13, color='gray', linestyle='--', alpha=0.3)
plt.axvline(x=26, color='gray', linestyle='--', alpha=0.3)
plt.axvline(x=39, color='gray', linestyle='--', alpha=0.3)
plt.text(6.5, plt.ylim()[1]*0.95, 'Q1', ha='center', fontsize=12, fontweight='bold', color='gray')
plt.text(19.5, plt.ylim()[1]*0.95, 'Q2', ha='center', fontsize=12, fontweight='bold', color='gray')
plt.text(32.5, plt.ylim()[1]*0.95, 'Q3', ha='center', fontsize=12, fontweight='bold', color='gray')
plt.text(45.5, plt.ylim()[1]*0.95, 'Q4', ha='center', fontsize=12, fontweight='bold', color='gray')

plt.legend(fontsize=11)
plt.tight_layout()
plt.savefig('time_series_weekly_pattern.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# 6. HOLIDAY IMPACT ANALYSIS
# ============================================================================

print("\n" + "-"*70)
print("6. HOLIDAY IMPACT ON SALES")
print("-"*70)

holiday_comparison = df.groupby('Holiday_Flag')['Weekly_Sales'].agg([
    ('Average', 'mean'),
    ('Median', 'median'),
    ('Std', 'std'),
    ('Count', 'count')
]).reset_index()
holiday_comparison['Holiday_Flag'] = holiday_comparison['Holiday_Flag'].map({0: 'Non-Holiday', 1: 'Holiday'})

print("\nHoliday vs Non-Holiday Comparison:")
print(holiday_comparison.to_string(index=False))

# Calculate impact
if len(holiday_comparison) == 2:
    non_holiday_avg = holiday_comparison[holiday_comparison['Holiday_Flag']=='Non-Holiday']['Average'].values[0]
    holiday_avg = holiday_comparison[holiday_comparison['Holiday_Flag']=='Holiday']['Average'].values[0]
    impact = ((holiday_avg - non_holiday_avg) / non_holiday_avg) * 100
    
    print(f"\n📊 HOLIDAY IMPACT:")
    print(f"   Non-Holiday Average: ${non_holiday_avg:,.2f}")
    print(f"   Holiday Average: ${holiday_avg:,.2f}")
    print(f"   Impact: {impact:+.2f}%")
    
    if impact > 0:
        print(f"   ✅ Holiday weeks boost sales by {abs(impact):.1f}%")
    else:
        print(f"   ⚠️ Holiday weeks decrease sales by {abs(impact):.1f}%")

# Visualize holiday impact
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Bar chart
axes[0].bar(holiday_comparison['Holiday_Flag'], holiday_comparison['Average'], 
            color=['#667eea', '#f5576c'], alpha=0.8, edgecolor='black', linewidth=2)
axes[0].set_title('Average Sales: Holiday vs Non-Holiday', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Average Sales ($)', fontsize=12)
axes[0].grid(axis='y', alpha=0.3)
for i, v in enumerate(holiday_comparison['Average']):
    axes[0].text(i, v, f'${v:,.0f}', ha='center', va='bottom', 
                fontweight='bold', fontsize=12)

# Box plot
df.boxplot(column='Weekly_Sales', by='Holiday_Flag', ax=axes[1])
axes[1].set_title('Sales Distribution: Holiday vs Non-Holiday', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Holiday Flag', fontsize=12)
axes[1].set_ylabel('Weekly Sales ($)', fontsize=12)
axes[1].set_xticklabels(['Non-Holiday', 'Holiday'])
plt.suptitle('')

plt.tight_layout()
plt.savefig('time_series_holiday_impact.png', dpi=300, bbox_inches='tight')
plt.show()

TIME SERIES ANALYSIS: WALMART SALES PATTERNS

Date Range: 2010-02-05 to 2012-10-26
Total Time Span: 994 days
Number of Weeks: 143
Number of Stores: 45

----------------------------------------------------------------------
1. OVERALL SALES TREND (ALL STORES)
----------------------------------------------------------------------

Trend Analysis:
  📈 UPWARD TREND: Sales are increasing over time
  Slope: $610.73 per day
  R² Score: 0.0011 (how well trend line fits)

----------------------------------------------------------------------
2. YEARLY SALES ANALYSIS
----------------------------------------------------------------------

Yearly Sales Summary:
 Year  Total_Sales  Average_Sales  Median_Sales     Std_Sales  Min_Sales  Max_Sales  Count
 2010 2.288886e+09   1.059670e+06    974696.055 582386.101284  209986.25 3818686.45   2160
 2011 2.448200e+09   1.046239e+06    948390.020 569773.443767  215359.21 3676388.98   2340
 2012 2.000133e+09   1.033660e+06    960476.100 536653.455829  236920

In [38]:
# Sales by Month and Holiday Flag
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Monthly sales split by holiday
monthly_holiday = df.groupby(['Month', 'Holiday_Flag'])['Weekly_Sales'].mean().unstack()
monthly_holiday.plot(kind='bar', ax=axes[0], color=['steelblue', 'coral'], width=0.8)
axes[0].set_title('Average Sales by Month and Holiday Flag', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Average Weekly Sales ($)')
axes[0].set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                         'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation=45)
axes[0].legend(['Non-Holiday', 'Holiday'])
axes[0].grid(axis='y', alpha=0.3)

# Quarterly sales by year
quarterly_year = df.groupby(['Quarter', 'Year'])['Weekly_Sales'].mean().unstack()
quarterly_year.plot(kind='bar', ax=axes[1], colormap='tab10', width=0.8)
axes[1].set_title('Average Sales by Quarter and Year', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Quarter')
axes[1].set_ylabel('Average Weekly Sales ($)')
axes[1].set_xticklabels(['Q1', 'Q2', 'Q3', 'Q4'], rotation=0)
axes[1].legend(title='Year')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [39]:
df.to_csv('walmart_cleaned.csv', index=False)
print("✓ Cleaned data saved successfully!")

# Create a summary report
summary = {
    'Total Records': len(df),
    'Date Range': f"{df['Date'].min()} to {df['Date'].max()}",
    'Number of Stores': df['Store'].nunique(),
    'Total Sales': df['Weekly_Sales'].sum(),
    'Average Weekly Sales': df['Weekly_Sales'].mean(),
    'Missing Values': df.isnull().sum().sum(),
    'Features': len(df.columns)
}

print("\n=== DATA SUMMARY ===")
for key, value in summary.items():
    print(f"{key}: {value}")

# Save summary to text file
with open('data_summary.txt', 'w') as f:
    f.write("Walmart Dataset - EDA Summary\n")
    f.write("=" * 50 + "\n\n")
    for key, value in summary.items():
        f.write(f"{key}: {value}\n")
        
print("\n✓ Summary report saved to 'data_summary.txt'")

✓ Cleaned data saved successfully!

=== DATA SUMMARY ===
Total Records: 6435
Date Range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
Number of Stores: 45
Total Sales: 6737218987.11
Average Weekly Sales: 1046964.8775617715
Missing Values: 0
Features: 15

✓ Summary report saved to 'data_summary.txt'
